# A3: A\*, IDS, and Effective Branching Factor

This notebook implements the Recursive Best-First Search implementation of the A* algorithm as well as the Iterative Deepening Search for comparing the effectiveness of each.  As a way to measure effectiveness, the code also includes a method that estimate of the effective branching factor for a search algorithm when applied to a search problem. Each algorithm will attempt to solve several eight-tile sliding puzzles.

Primary functions used for general searching:

   - `aStarSearch(startState, actionsF, takeActionF, goalTestF, hF)`
   - `iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth)`
   - `ebf(nNodes, depth, precision=0.01)`

Secondary functions used for 8 puzzle searching:

  * `actionsF_8p(state)`
  * `takeActionF_8p(state, action)`
  * `goalTestF_8p(state, goal)`
  
Tertiary function used to print results in a table

   - runExperiment(goalState1, goalState2, goalState3, [h1, h2, h3, ...])

## Heuristic Functions

The Recursive Iterative Deepening form of the A* algorithm was taken from the lecture notes on Informed Search. It will use the following heuristic functions to estimate the best, most informed move.

  * `h1_8p(state, goal)`: $h(state, goal) = 0$, for all states $state$ and all goal states $goal$,
  * `h2_8p(state, goal)`: $h(state, goal) = m$, where $m$ is the Manhattan distance that the blank is from its goal position,
  * `h3_8p(state, goal)`: $h(state, goal) = m$, where $m$ is the Manhattan distance of all points from their goal position,
  * `h4_8p(state, goal)`: $h(state, goal) = m$, where $m$ is the total of tiles not in their goal position
  * `h5_8p(state, goal)`: $h(state, goal) = m$, where $m$ is the total of tiles not in their final goal column or row

## Comparison

Each function will be run with the following start state

$$
\begin{array}{ccc}
1 & 2 & 3\\
4 & 0 & 5\\
6 & 7 & 8
\end{array}
$$

and these three goal states.

$$
\begin{array}{ccccccccccc}
1 & 2 & 3  & ~~~~ & 1 & 2 & 3  &  ~~~~ & 1 & 0 &  3\\
4 & 0 & 5  & & 4 & 5 & 8  & & 4 & 5 & 8\\
6 & 7 & 8 &  & 6 & 0 & 7  & & 2 & 6 & 7
\end{array}
$$

The format and expected values are below.  Final results will include several more heuristic funtcions


           [1, 2, 3, 4, 0, 5, 6, 7, 8]    [1, 2, 3, 4, 5, 8, 6, 0, 7]    [1, 0, 3, 4, 5, 8, 2, 6, 7] 
    Algorithm    Depth  Nodes  EBF              Depth  Nodes  EBF              Depth  Nodes  EBF          
         IDS       0      0  0.000                3     43  3.086               11 225850  2.954         
        A*h1       0      0  0.000                3    116  4.488               11 643246  3.263         
        A*h2       0      0  0.000                3     51  3.297               11 100046  2.733         

Below is the main code block for the notebook

In [ ]:
import copy
import pandas as pd
from collections import defaultdict
#  Recursive Iterative Deepening form of A* taken from Informed Search

NODES = 0


class Node:
    def __init__(self, state, f=0, g=0, h=0):
        self.state = state
        self.f = f
        self.g = g
        self.h = h

    def __repr__(self):
        return "Node(" + repr(self.state) + ", f=" + repr(self.f) + \
               ", g=" + repr(self.g) + ", h=" + repr(self.h) + ")"

# REQUIRED
def aStarSearch(startState, actionsF, takeActionF, goalTestF, hF):
    global NODES
    NODES = 0
    h = hF(startState)
    startNode = Node(state=startState, f=0 + h, g=0, h=h)
    return aStarSearchHelper(startNode, actionsF, takeActionF, goalTestF, hF, float('inf'))


def aStarSearchHelper(parentNode, actionsF, takeActionF, goalTestF, hF, fmax):
    if goalTestF(parentNode.state):
        return [parentNode.state], parentNode.g
    # Construct list of children nodes with f, g, and h values
    actions = actionsF(parentNode.state)
    if not actions:
        return "failure", float('inf')
    children = []
    for action in actions:
        (childState, stepCost) = takeActionF(parentNode.state, action)
        h = hF(childState)
        g = parentNode.g + stepCost
        f = max(h+g, parentNode.f)
        childNode = Node(state=childState, f=f, g=g, h=h)
        children.append(childNode)
    while True:
        # find best child
        children.sort(key = lambda n: n.f) # sort by f value
        bestChild = children[0]
        if bestChild.f > fmax:
            return "failure", bestChild.f
        # next lowest f value
        alternativef = children[1].f if len(children) > 1 else float('inf')
        # expand best child, reassign its f value to be returned value
        result, bestChild.f = aStarSearchHelper(bestChild, actionsF, takeActionF, goalTestF, hF, min(fmax, alternativef))
        if result is not "failure":
            result.insert(0, parentNode.state)
            return result, bestChild.f


# REQUIRED
def iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth):
    global NODES
    NODES = 0
    for depth in range(maxDepth):
        result = depthLimitedSearch(startState, goalState, actionsF, takeActionF, depth)
        # Failure to find goal
        if result == 'failure':
            return 'failure'
        # Solution as list of States
        if result != 'cutoff':
            result.insert(0, startState)
            return result
    # Cutoff
    return 'cutoff'


def depthLimitedSearch(startState, goalState, actionsF, takeActionF, depthLimit):
    if startState == goalState:
        return []
    if depthLimit == 0:
        return 'cutoff'
    cutoffOccurred = False
    for action in actionsF(startState):
        state = startState
        childState, temp = takeActionF(state, action)
        result = depthLimitedSearch(childState, goalState, actionsF, takeActionF, depthLimit-1)
        if result == 'cutoff':
            cutoffOccurred = True
        # Solution as list of States
        elif result != 'failure':
            result.insert(0, childState)
            return result
    # Cutoff
    if cutoffOccurred:
        return 'cutoff'
    # Failure to find goal
    else:
        return 'failure'


# Required: Returns estimate based on the number of nodes and the depth
def ebf(nNodes, depth, precision=0.01):
    
    if nNodes == 0:
        return float(0)
    elif depth == 0:
        return float(1)
    elif nNodes == 1 and depth == 1:
        return 2
    else:
        return recursiveBinary(nNodes, depth, precision, 0, depth**2)


def recursiveBinary(nNodes, depth, precision, lo, hi):
    mid = (hi + lo) / 2
    
    if mid == 1:
        estimate = 1 + mid + mid**2
    else:
        estimate = (1-mid**(depth+1))/(1-mid)

    if (nNodes - precision) <= estimate <= (nNodes + precision):
        return mid
    elif estimate > nNodes:
        return recursiveBinary(nNodes, depth, precision, lo, mid)
    elif estimate < nNodes:
        return recursiveBinary(nNodes, depth, precision, mid, hi)
    else:
        return -1


def goalTestF_8p(state, goal):
    return state == goal


# Required: Return a list of up to four valid actions that can be applied in state. Order is left, right, up, down
def actionsF_8p(currentState):
    x, y = findBlank_8p(currentState)
    moves = []
    if x != 0:
        moves.append(('left', 1))
    if x != 2:
        moves.append(('right', 1))
    if y != 0:
        moves.append(('up', 1))
    if y != 2:
        moves.append(('down', 1))
    return moves


# REquired: Return the state that results from applying action in state.
def takeActionF_8p(currentState, command):
    global NODES
    NODES += 1
    
    index = currentState.index(0)
    actions = actionsF_8p(currentState)
    nextState = currentState.copy()
    direction, step = command
    actionsDict = dict(actions)

    if direction == 'left':
        nextState[index - 1], nextState[index] = nextState[index], nextState[index - 1]
    elif direction == 'right':
        nextState[index + 1], nextState[index] = nextState[index], nextState[index + 1]
    elif direction == 'up':
        nextState[index - 3], nextState[index] = nextState[index], nextState[index - 3]
    elif direction == 'down':
        nextState[index + 3], nextState[index] = nextState[index], nextState[index + 3]
    else:
        print('Error: Illegal Move')
    
    return nextState, step


# Return the row and column index for the location of all numbers as a dict
def findNumbers_8p(currentState):
    positions = {}
    for key in currentState:
        positions[key] = coordinate(currentState.index(key), 3)
    return positions


# Return the row and column index for the location of the blank
def findBlank_8p(currentState):
    index = currentState.index(0)
    return coordinate(index, 3)


def coordinate(index, width):
    x = index % width
    y = index // width
    return x, y


def manhattanDistance(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    return abs(x2 - x1) + abs(y2 - y1)
    # return sum(abs(e - s) for s,e in zip(start, end))


# Heuristic equal to zero
def h1_8p(state, goal):
    return 0;


# Manhattan distance of blank
def h2_8p(state, goal):
    return manhattanDistance(findBlank_8p(state), findBlank_8p(goal))


# Manhattan distance of all points
def h3_8p(state, goal):
    distance = 0
    stateDict = findNumbers_8p(state)
    # print(stateDict)
    goalDict = findNumbers_8p(goal)
    for key in stateDict:
        distance += manhattanDistance(stateDict[key], goalDict[key])
    return distance


# Displaced tile count
def h4_8p(state, goal):
    count = 0
    for index in state:
        if state[index] != goal[index]:
            count += 1
    return count


# Displaced tile in row or column count
def h5_8p(state, goal):
    count = 0
    for index in state:
        if state[index] != goal[index]:
            x1, y1 = coordinate(state[index], 3)
            x2, y2 = coordinate(goal.index(state[index]), 3)
            if x1 != y1:
                count += 1
            elif x2 != y2:
                count += 1
    return count


def runExperiment(goalState1, goalState2, goalState3, functions):

    columns = ('Algorithm', 'Depth', 'Nodes', 'EBF', 'Depth', 'Nodes', 'EBF', 'Depth', 'Nodes', 'EBF')
    raw_data = defaultdict(list)

    print(' {} {} {}'.format(goalState1, goalState2, goalState3))
    state = [goalState1, goalState2, goalState3]
    start = [1, 2, 3, 4, 0, 5, 6, 7, 8]
    i = 0
    raw_data[i].append('IDS')
    for goal in state:
        ids = iterativeDeepeningSearch(start, goal, actionsF_8p, takeActionF_8p, 100)
        depth = len(ids) - 1
        raw_data[i].extend([depth, NODES, format(ebf(NODES, depth), '.2f')])

    for heuristic in functions:
        i += 1
        raw_data[i].append('A*h{}'.format(i))
        for goal in state:
            pathList, depth = aStarSearch(start, actionsF_8p, takeActionF_8p, lambda state: goalTestF_8p(state, goal), lambda state: heuristic(state, goal))
            raw_data[i].extend([depth, NODES, format(ebf(NODES, depth), '.2f')])

    df = pd.DataFrame.from_dict(raw_data, 'index')
    df.columns = columns
    print(df.to_string(index=False))

Main method to execute code above and display results

In [ ]:
    goalState1 = [1, 2, 3, 4, 0, 5, 6, 7, 8]
    goalState2 = [1, 2, 3, 4, 5, 8, 6, 0, 7]
    goalState3 = [1, 0, 3, 4, 5, 8, 2, 6, 7]
    hfs = [h1_8p, h2_8p, h3_8p, h4_8p, h5_8p]
    runExperiment(goalState1, goalState2, goalState3, hfs)

Example output for the effective branching function. Parameter are nodes and depth

In [ ]:
ebf(10, 3)

In [ ]:
ebf(1, 0)

In [ ]:
ebf(2, 1)

In [ ]:
ebf(2, 1, precision=0.000001)

In [ ]:
ebf(200000, 5)

In [ ]:
ebf(200000, 50)

Simple test showing an Iterative Deepening Search through a tree and the output of the various functions.

In [ ]:
def actionsF_simple(state):
    succs = {'a': ['b', 'c'], 'b':['a'], 'c':['h'], 'h':['i'], 'i':['j', 'k', 'l'], 'k':['z']}
    return [(s, 1) for s in succs.get(state, [])]

def takeActionF_simple(state, action):
    return action

def goalTestF_simple(state, goal):
    return state == goal

def h_simple(state, goal):
    return 1

In [ ]:
actions = actionsF_simple('a')
actions

In [ ]:
takeActionF_simple('a', actions[0])

In [ ]:
goalTestF_simple('a', 'a')

In [ ]:
h_simple('a', 'z')

In [ ]:
iterativeDeepeningSearch('a', 'z', actionsF_simple, takeActionF_simple, 10)

In [ ]:
aStarSearch('a',actionsF_simple, takeActionF_simple,
            lambda s: goalTestF_simple(s, 'z'),
            lambda s: h_simple(s, 'z'))

Code and Execution of the Autograder

In [ ]:
def grader():
    g = 0;
    print('\nTesting actionsF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8])')
    acts = actionsF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8])
    correct = [('left', 1), ('right', 1), ('up', 1)]
    if acts == correct:
        g += 5
        print('\n--- 5/5 points. Your actionsF_8p correctly returned', acts)
    else:
        print('\n--- 0/5 points. Your actionsF_8p should have returned', correct, 'but you returned', acts)

    print('\nTesting takeActionF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], (''up'', 1))')
    s = takeActionF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], ('up', 1))
    correct = ([1, 2, 3, 4, 0, 6, 7, 5, 8], 1)
    if s == correct:
        g += 5
        print('\n--- 5/5 points. Your takeActionsF_8p correctly returned', s)
    else:
        print('\n--- 0/5 points. Your takeActionsF_8p should have returned', correct, 'but you returned', s)

    print('\nTesting goalTestF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 5, 6, 7, 0, 8])')
    if goalTestF_8p([1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 5, 6, 7, 0, 8]):
        g += 5
        print('\n--- 5/5 points. Your goalTestF_8p correctly True')
    else:
        print('\n--- 0/5 points. Your goalTestF_8p did not return True')

    print('\nTesting aStarSearch([1, 2, 3, 4, 5, 6, 7, 0, 8],')
    print('                     actionsF_8p, takeActionF_8p,')
    print('                     lambda s: goalTestF_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]),')
    print('                     lambda s: h1_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]))')

    path = aStarSearch([1, 2, 3, 4, 5, 6, 7, 0, 8], actionsF_8p, takeActionF_8p,
                       lambda s: goalTestF_8p(s, [0, 2, 3, 1, 4, 6, 7, 5, 8]),
                       lambda s: h1_8p(s, [0, 2, 3, 1, 4, 6, 7, 5, 8]))

    correct = ([[1, 2, 3, 4, 5, 6, 7, 0, 8], [1, 2, 3, 4, 0, 6, 7, 5, 8], [1, 2, 3, 0, 4, 6, 7, 5, 8],
                [0, 2, 3, 1, 4, 6, 7, 5, 8]], 3)
    if path == correct:
        g += 20
        print('\n--- 20/20 points. Your search correctly returned', path)
    else:
        print('\n---  0/20 points. Your search should have returned', correct, 'but you returned', path)

    print('\nTesting iterativeDeepeningSearch([5, 2, 8, 0, 1, 4, 3, 7, 6], ')
    print('                                 [0, 2, 3, 1, 4,  6, 7, 5, 8],')
    print('                                 actionsF_8p, takeActionF_8p, 10)')
    path = iterativeDeepeningSearch([5, 2, 8, 0, 1, 4, 3, 7, 6], [0, 2, 3, 1, 4, 6, 7, 5, 8], actionsF_8p, takeActionF_8p, 10)
    if type(path) == str and path.lower() == 'cutoff':
        g += 15
        print('\n--- 15/15 points. Your search correctly returned', path)
    else:
        print('\n---  0/15 points. Your search should have returned ''cutoff'', but you returned', path)

    print('\nTotal is {}/50'.format(g))

In [ ]:
grader()